In [1]:
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests
import numpy as np
from datetime import datetime

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 


Source_data = pd.read_csv("/mnt/c/Users/ralvin/Downloads/837I00111182024.csv")



Provider_TIN_dict = {'MAPPINGS':{'TIN':['Name', 'Unique_TIN__c'], 'Address':['Address__c'], 'City':['City__c'], 'State':['State__c'], 'Zip':['Zip__c'],
                      'Provider':['Provider_Name__c']}, 'KEYS':{'Provider_TIN__c':'Unique_TIN__c'}}

Account_dict = {'MAPPINGS':{'TIN':['Tax_ID__c'], 'BillProvAddress1':['BillingAddress'], 'BillProvCity':['BillingCity'], 'BillProvState':['BillingState'],
                 'BillProvZip':['BillingPostalCode'], 'Provider':['Name']}, 'KEYS':{'Account':'Tax_ID__c'}}



Claim_dict = {'MAPPINGS':{'Claim ID':['Claim_ID__c'], 'TIN':['Provider_TIN__c'], 'Patient Group/Policy Number':['Group_Client__c'], 'Claim Number':['Claim_Number__c'],'JurisdictionState':['Jurisdiction__c'],
               'Diag Code 1st':['Diag_Code__c'],'Diag Code 2nd':['Diag_Code_2nd__c'], 'Diag Code 3rd':['Diag_Code_3rd__c'], 'Diag Code 4th':['Diag_Code_4th__c'], 'Provider':['Provider__c'],
               'Patient':['Patient__c'], 'Patient ID':['Patient_ID__c'], 'Date of Birth':['Date_of_Birth__c'],'Zip':['Provider_Zip__c'], 'DRG':['DRG__c'], 'IsQPA':['QPA__c'], 'NPI':['NPI__c'],
               'Begin DOS':['Begin_DOS__c'], 'End DOS':['End_DOS__c'], 'PlaceofService':['POS__c'], 'Billing Provider Taxonomy':['Provider_Specialty__c', 'Service_Type__c']}, 'KEYS':{'Claims__c':'Claim_ID__c'}, 
               'TABLE_CONVERT':{'TIN':{'Provider_TIN__c':['Name', 'Id']}, 'Patient Group/Policy Number':{'Groups_Clients__c':['Group_Number__c', 'Id']}, 'JurisdictionState':{'Jurisdiction__c':['Name', 'Id']},
                                 'DRG':{'DRG__c':['Name', 'Id']}, 'Billing Provider Taxonomy':{'Provider_Specialty__c':['Name', 'Id', 'Service_Type__c']}},
                'KEYS':{'Claims__c':'Claim_ID__c'}
               }

Line_item_dict = {'MAPPINGS':{'Begin DOS':['DOS__c'], 'Revenue Code':['Rev_Code__c'],'Billed Amount':['Billed_Amount__c'], 'Claim ID':['RHP_ID__c'], 'Mod':['Mod__c'],
                   'Mod (2nd)':['Mod_2nd__c'], 'Units':['Units__c'], 'MAR':['MAR__c'], 'HCPCS/CPT Code':['HCPCS_CPT_Code__c'], 'Line ID':['Line_ID__c'], 
                    '4LCodes':['Exp_Code_2nd__c']},
                  'KEYS':{'Line_Items__c':'RHP_ID__c'},
                  'TABLE_CONVERT':{'Claim ID':{'Claims__c':['Claim_ID__c']}, 'HCPCS/CPT Code':{'HCPCS_CPT_Code__c':['Name', 'Id']}}
                  }





def salesforce_connection(sandbox=True):
    session = requests.Session()
    security_token = 'pIjwc8P0OB2ZisbQYhlKxfLd'
    username = 'ralvin@relianthp.com.tctdev'
    password = 'Flash363!'
    domain = 'test'

    session_id, instance = SalesforceLogin(username=username, password=password, security_token=security_token, domain=domain)
    sf = Salesforce(instance=instance, session_id=session_id, session=session)
    print('Connected to Salesforce (Sandbox)')
    return sf

sf = salesforce_connection()

all_objects = sf.describe()["sobjects"]


accounts_metadata = sf.Account.describe()
account_cols = [field['name'] for field in accounts_metadata['fields']]

provider_tin_metadata = sf.Provider_TIN__c.describe()
provider_tin_cols = [field['name'] for field in provider_tin_metadata['fields']]

claims_metadata = sf.Claims__c.describe()
claim_cols = [field['name'] for field in claims_metadata['fields']]

line_item_metadata = sf.Line_Items__c.describe()
line_item_cols = [field['name'] for field in line_item_metadata['fields']]

clients_metadata = sf.Clients__c.describe()
clients_cols = [field['name'] for field in clients_metadata['fields']]

group_clients_metadata = sf.Groups_Clients__c.describe()
group_clients_cols = [field['name'] for field in group_clients_metadata['fields']]

jurisdiction_metadata = sf.Jurisdiction__c.describe()
jurisdiction_cols = [field['name'] for field in jurisdiction_metadata['fields']]

drg_metadata = sf.DRG__c.describe()
drg_cols = [field['name'] for field in drg_metadata['fields']]

hcpcs_cpt_metadata = sf.HCPCS_CPT_Code__c.describe()
hcpcs_cpt_cols = [field['name'] for field in hcpcs_cpt_metadata['fields']]

provider_specialty_metadata = sf.Provider_Specialty__c.describe()
provider_specialty_cols = [field['name'] for field in provider_specialty_metadata['fields']]

needed_objects = { "Claims__c": claim_cols, "Line_Items__c": line_item_cols, "Provider_TIN__c": provider_tin_cols, 
                 'Jurisdiction__c': jurisdiction_cols, 'Groups_Clients__c': group_clients_cols, 'DRG__c': drg_cols, 
                 'HCPCS_CPT_Code__c': hcpcs_cpt_cols, 'Account':account_cols, 'Clients__c': clients_cols, 'Provider_Specialty__c':provider_specialty_cols}


DATA_MAPPINGS = {'Provider_Object_Map':Provider_TIN_dict, 'Account_Object_Map':Account_dict, 'Claim_Object_Map':Claim_dict, 'Line_Item_Object_Map':Line_item_dict}




table_store = {}

for object_name in needed_objects:
    print(object_name)

    object_fields = needed_objects[object_name]

    fields_str = ", ".join(object_fields)
    query = f"SELECT {fields_str} FROM {object_name}"
    results = sf.query_all(query)
    records = results['records']
    df = pd.DataFrame(records).drop(columns='attributes')
    print(df.shape)
    table_store[object_name] = df

Connected to Salesforce (Sandbox)
Claims__c
(13033, 100)
Line_Items__c
(22789, 48)
Provider_TIN__c
(10853, 27)
Jurisdiction__c
(53, 16)
Groups_Clients__c
(1702, 58)
DRG__c
(808, 20)
HCPCS_CPT_Code__c
(10045, 36)
Account
(2217, 98)
Clients__c
(62, 58)
Provider_Specialty__c
(870, 18)


In [17]:
sf_claim_table = table_store['Claims__c']
sf_claim_table[sf_claim_table['Claim_ID__c'] == '317D671330C4442tksft']

,Id,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,LastViewedDate,LastReferencedDate,Duplicate__c,Client__c,Claim_Number__c,Jurisdiction__c,Provider_TIN__c,Contains_Line_Items__c,Status__c,Appeal__c,Date_Fee_Paid__c,Fee_Credit_Date__c,Diag_Code__c,Diag_Code_2nd__c,Diag_Code_3rd__c,Proc_Code__c,Proc_Code_2nd__c,Diag_Code_4th__c,Total_Billed__c,Provider__c,Group_Client__c,Claim_ID__c,Received_Date__c,Appeal_Rec_d_Date__c,Last_Contact__c,Appeal_Stage__c,State_Appeal__c,Contact__c,Phone__c,Fax__c,Email__c,Notes__c,Appeal_Closed_Date__c,POS__c,Attorney_Assigned__c,Detailed_Appeal_Resp__c,Begin_DOS__c,End_DOS__c,Total_MAR__c,Total_Allowed__c,Total_Reductions__c,Fee__c,Patient__c,Date_of_Birth__c,Patient_ID__c,Total_Additional_Payment__c,Fee_Credit__c,Savings__c,Provider_Zip__c,DRG__c,Pended_Token__c,Client_Id__c,Service_Type__c,PHS_Id__c,Total_Reconsidered__c,X3_Digit_Zip__c,Prime_Return_Filename__c,Contact_Name__c,Proposed_Settlement__c,Proposed_Additional__c,Paid_Fee__c,Gross_Fee__c,QPA__c,Submitted__c,Provider_Specialty__c,Exp_Code_Test__c,Exp_Code_Test2__c,Pricing_Method__c,PEPM_Fee__c,Total_QPA_Allowed__c,NPI__c,SLA__c,Created_Date_Formula__c,Balance_Bill__c,Contract__c,Producer__c,Payer__c,Company__c,API_Status__c,API_Confirmation__c,Total_Savings__c,Bill_Type__c,Form_Type__c,Provider_Account__c,Original_Claim_for_Reconsideration__c,Return_Date_Deadline__c,Savings_Threshold__c,Total_Code_Edit__c,Original_Claim_from_Duplication__c,Recon_Dupe__c,Blank_Allowed_Count__c
12954,a06VF000009x0ejYAA,False,1079851,2024-12-12T21:55:48.000+0000,005VF00000AhMNjYAN,2024-12-12T21:55:51.000+0000,005VF00000AhMNjYAN,2024-12-13T14:20:38.000+0000,None,2024-12-12T21:55:51.000+0000,2024-12-12T21:55:51.000+0000,1984-03-126104617530,a048a00000iOnmHAAS,U424319281216,a024W00000HYAjcQAH,a0FVF000001f3W22AI,False,To be Repriced,False,None,None,Z1211,K635,K648,None,None,K2950,0.0,DEACONESS HENDERSON HOSPII,a0CUu000001thoWMAQ,317D671330C4442tksft,2024-12-12,None,None,None,False,None,None,None,None,None,None,13,False,None,None,None,0.0,0.0,0.0,0.0,COBB BETHAN,1984-03-12,FBDS11454,0.0,0.0,NaN,452638704,None,None,None,OUTPATIENT,None,0.0,452,REL_,None,NaN,0.0,None,0.0,N,None,a008a000015yXODAA2,None,None,FMP,NaN,None,1295211563,NaN,2024-12-12T21:55:48.000+0000,False,None,None,None,None,None,None,NaN,None,HCFA,None,None,None,NaN,0.0,None,None,0.0


In [3]:
# print((map_claims_table.shape, len(set(map_claims_table['Claim_ID__c']))))


# claim_ids = []
# for i in range(len(map_claims_table)):
#     record = map_claims_table.loc[i, 'Claim_ID__c']
#     claim_ids.append(record)
# len(set(claim_ids))


sf_claim_table = table_store['Claims__c']
sf_claim_table[sf_claim_table['Claim_ID__c'] == 'A3038B5E4B6C914tksft']

duplicate_rows = sf_claim_table[sf_claim_table.duplicated(subset='Claim_ID__c', keep=False)]
print(duplicate_rows.shape)

# duplicate_rows[duplicate_rows['Claim_ID__c'] == '10005410']
# duplicate_rows[duplicate_rows['Claim_ID__c'] == '12720823']
duplicate_rows[duplicate_rows['Claim_ID__c'] == 'B099995A56B4ADAtksft']




(220, 100)


,Id,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,LastViewedDate,LastReferencedDate,Duplicate__c,Client__c,Claim_Number__c,Jurisdiction__c,Provider_TIN__c,Contains_Line_Items__c,Status__c,Appeal__c,Date_Fee_Paid__c,Fee_Credit_Date__c,Diag_Code__c,Diag_Code_2nd__c,Diag_Code_3rd__c,Proc_Code__c,Proc_Code_2nd__c,Diag_Code_4th__c,Total_Billed__c,Provider__c,Group_Client__c,Claim_ID__c,Received_Date__c,Appeal_Rec_d_Date__c,Last_Contact__c,Appeal_Stage__c,State_Appeal__c,Contact__c,Phone__c,Fax__c,Email__c,Notes__c,Appeal_Closed_Date__c,POS__c,Attorney_Assigned__c,Detailed_Appeal_Resp__c,Begin_DOS__c,End_DOS__c,Total_MAR__c,Total_Allowed__c,Total_Reductions__c,Fee__c,Patient__c,Date_of_Birth__c,Patient_ID__c,Total_Additional_Payment__c,Fee_Credit__c,Savings__c,Provider_Zip__c,DRG__c,Pended_Token__c,Client_Id__c,Service_Type__c,PHS_Id__c,Total_Reconsidered__c,X3_Digit_Zip__c,Prime_Return_Filename__c,Contact_Name__c,Proposed_Settlement__c,Proposed_Additional__c,Paid_Fee__c,Gross_Fee__c,QPA__c,Submitted__c,Provider_Specialty__c,Exp_Code_Test__c,Exp_Code_Test2__c,Pricing_Method__c,PEPM_Fee__c,Total_QPA_Allowed__c,NPI__c,SLA__c,Created_Date_Formula__c,Balance_Bill__c,Contract__c,Producer__c,Payer__c,Company__c,API_Status__c,API_Confirmation__c,Total_Savings__c,Bill_Type__c,Form_Type__c,Provider_Account__c,Original_Claim_for_Reconsideration__c,Return_Date_Deadline__c,Savings_Threshold__c,Total_Code_Edit__c,Original_Claim_from_Duplication__c,Recon_Dupe__c,Blank_Allowed_Count__c
10000,a06VF000005Lb1hYAC,False,1076717,2024-05-03T16:34:34.000+0000,0058a00000N3AI9AAN,2024-06-06T15:18:34.000+0000,0058a00000N3AI9AAN,2024-12-13T14:20:38.000+0000,None,None,None,2023-06-021965-12-2835199999974.4,a048a00000qJMlIAAW,999ACZATZR1,a024W00000HYAjJQAX,a0FVF000000b3oX2AQ,True,To be Repriced,False,None,None,I480,I4892,E6601,None,None,I471,74.4,"INDIANA UNIVERSITY, HEALTH",a0CVF00000UXLZY2A5,B099995A56B4ADAtksft,2024-05-03,None,None,None,False,None,None,None,None,None,None,None,False,None,2023-06-02,2023-06-02,80.4,0.0,80.4,0.0,"patient, name",1965-12-28,QWCR19999,0.0,0.0,100.0,None,None,None,None,PHYSICIAN,None,0.0,None,REL_,None,NaN,0.0,None,0.0,N,None,a008a000015yXODAA2,None,None,WORK COMP,NaN,None,3331343,36.0,2024-05-03T16:34:34.000+0000,False,800VF000004cI85YAE,0018a00002H0uJVAAZ,0018a00002H0uJVAAZ,001VF0000061TpJYAU,None,None,100.0,WORKERS COMP,HCFA,001VF0000061ULZYA2,None,None,NaN,0.0,None,None,6.0
10012,a06VF00000664yNYAQ,False,1076731,2024-06-04T17:47:23.000+0000,0058a00000N3AI9AAN,2024-06-04T18:34:18.000+0000,0058a00000N3AI9AAN,2024-12-13T14:20:38.000+0000,None,None,None,2023-06-021965-12-2835199999924.8,a048a00000qJMlIAAW,999ACZATZR1,a024W00000HYAjJQAX,a0FVF000000b3oX2AQ,True,To be Repriced,False,None,None,I480,I4892,E6601,None,None,I471,24.8,"INDIANA UNIVERSITY, HEALTH",a0CVF00000cOA0l2AG,B099995A56B4ADAtksft,2024-06-04,None,None,None,False,None,None,None,None,None,None,None,False,None,2023-06-02,2023-06-02,26.8,0.0,26.8,0.0,"patient, name",1965-12-28,None,0.0,0.0,100.0,None,None,None,None,PHYSICIAN,None,0.0,None,REL_,None,NaN,0.0,None,0.0,N,None,None,None,None,WORK COMP,NaN,None,3331343,36.0,2024-06-04T17:47:23.000+0000,False,800VF000005LojhYAC,0018a00002H0uJVAAZ,0018a00002H0uJVAAZ,001VF0000061TpJYAU,None,None,100.0,WORKERS COMP,HCFA,001VF0000061ULZYA2,None,None,NaN,0.0,None,None,2.0
10016,a06VF0000068hsMYAQ,False,1076735,2024-06-06T15:20:24.000+0000,0058a00000N3AI9AAN,2024-06-06T15:25:12.000+0000,0058a00000N3AI9AAN,2024-12-13T14:20:38.000+0000,None,None,None,2023-06-021965-12-2835199999924.8,a04Uu0000004LkDIAU,999ACZATZR1,a024W00000HYAjJQAX,a0FVF000000b3oX2AQ,True,To be Repriced,False,None,None,I480,I4892,E6601,None,None,I471,24.8,"INDIANA UNIVERSITY, HEALTH",a0CVF00000ckSNh2AM,B099995A56B4ADAtksft,2024-06-06,None,None,None,False,None,None,None,None,None,None,None,False,None,2023-06-02,2023-06-02,26.8,0.0,26.8,0.0,"patient, name",1965-12-28,QWCR19999,0.0,0.0,100.0,None,None,None,None,P

In [14]:

sf_lines_table = table_store['Line_Items__c']
sf_lines_table[sf_lines_table['RHP_ID__c'] == 'a06VF000005Lb1hYAC']


,Id,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastViewedDate,LastReferencedDate,DOS__c,Rev_Code__c,Line_Number__c,Billed_Amount__c,RHP_ID__c,Mod__c,Mod_2nd__c,Units__c,MAR__c,Allowed__c,Client__c,Reconsideration__c,Savings__c,Reductions__c,Bill_Type__c,Exp_Desc__c,HCPCS_CPT_Code__c,Additional_Payment__c,Line_ID__c,Pricing_Method__c,Provider_TIN_Claim__c,Jurisdiction_Claim__c,Pricing_Trigger__c,Exp_Code__c,Exp_Code_2nd__c,MO_QPA_Allowed__c,Exp_Desc_2nd__c,MO_QPA_Reductions__c,MO_Fac_QPA_Reductions__c,MO_Fac_QPA_Allowed__c,MO_Anes_QPA__c,QPA_Allowed__c,MO_Anes_QPA_Reductions__c,QPA_Reductions__c,Unique_Line_Id__c,Code_Edit__c,NDC_Number__c,Target_Amount__c
10000,a0YVF000000OdRd2AK,False,L2294018,2024-05-03T16:34:34.000+0000,0058a00000N3AI9AAN,2024-06-06T15:18:34.000+0000,0058a00000N3AI9AAN,2024-06-06T15:18:34.000+0000,None,None,2023-06-02,483,None,12.4,a06VF000005Lb1hYAC,SA,25,10.0,13.4,NaN,a048a00000qJMlI,None,100.0,13.4,WORKERS COMP,None,a0b4W00000y1ISnQAM,None,1.0,WORK COMP,a0FVF000000b3oX2AQ,a024W00000HYAjJ,False,None,None,3699.20,None,-3686.80,-11912.90,11925.30,0.0,NaN,12.4,12.4,54178,NaN,None,NaN
10001,a0YVF000000OdRe2AK,False,L2294019,2024-05-03T16:34:34.000+0000,0058a00000N3AI9AAN,2024-06-06T15:18:34.000+0000,0058a00000N3AI9AAN,2024-06-06T15:18:34.000+0000,None,None,2023-06-02,483,None,12.4,a06VF000005Lb1hYAC,None,None,1.0,13.4,NaN,a048a00000qJMlI,None,100.0,13.4,WORKERS COMP,None,a0b4W00000y1ISnQAM,None,2.0,WORK COMP,a0FVF000000b3oX2AQ,a024W00000HYAjJ,False,None,None,369.92,None,-357.52,-1180.13,1192.53,0.0,NaN,12.4,12.4,54179,NaN,None,NaN
10086,a0YVF000000Z6492AC,False,L2294112,2024-06-04T06:32:27.000+0000,0058a00000N3AI9AAN,2024-06-04T06:32:27.000+0000,0058a00000N3AI9AAN,2024-06-04T06:32:27.000+0000,None,None,2023-06-02,483,None,12.4,a06VF000005Lb1hYAC,None,None,10.0,13.4,NaN,a048a00000qJMlI,None,100.0,13.4,WORKERS COMP,None,a0b4W00000y1ISnQAM,None,NaN,WORK COMP,a0FVF000000b3oX2AQ,a024W00000HYAjJ,False,None,None,3699.20,None,-3686.80,-11912.90,11925.30,0.0,NaN,12.4,12.4,None,NaN,None,NaN
10087,a0YVF000000Z64A2AS,False,L2294113,2024-06-04T06:32:27.000+0000,0058a00000N3AI9AAN,2024-06-04T06:32:27.000+0000,0058a00000N3AI9AAN,2024-06-04T06:32:27.000+0000,None,None,2023-06-02,483,None,12.4,a06VF000005Lb1hYAC,None,None,10.0,13.4,NaN,a048a00000qJMlI,None,100.0,13.4,WORKERS COMP,None,a0b4W00000y1ISnQAM,None,NaN,WORK COMP,a0FVF000000b3oX2AQ,a024W00000HYAjJ,False,None,None,3699.20,None,-3686.80,-11912.90,11925.30,0.0,NaN,12.4,12.4,None,NaN,None,NaN
10088,a0YVF000000Z65l2AC,False,L2294114,2024-06-04T06:32:57.000+0000,0058a00000N3AI9AAN,2024-06-06T09:09:16.000+0000,0058a00000N3AI9AAN,2024-06-06T09:09:16.000+0000,None,None,2023-06-02,483,None,12.4,a06VF000005Lb1hYAC,None,None,10.0,13.4,NaN,a048a00000qJMlI,None,100.0,13.4,WORKERS COMP,None,a0b4W00000y1ISnQAM,None,54178.0,WORK COMP,a0FVF000000b3oX2AQ,a024W00000HYAjJ,False,None,None,3699.20,None,-3686.80,-11912.90,11925.30,0.0,NaN,12.4,12.4,1,NaN,None,NaN
10089,a0YVF000000Z65m2AC,False,L2294115,2024-06-04T06:32:57.000+0000,0058a00000N3AI9AAN,2024-06-06T09:09:16.000+0000,0058a00000N3AI9AAN,2024-06-06T09:09:16.000+0000,None,None,2023-06-02,483,None,12.4,a06VF000005Lb1hYAC,None,None,10.0,13.4,NaN,a048a00000qJMlI,None,100.0,13.4,WORKERS COMP,None,a0b4W00000y1ISnQAM,None,54179.0,WORK COMP,a0FVF000000b3oX2AQ,a024W00000HYAjJ,False,None,None,3699.20,None,-3686.80,-11912.90,11925.30,0.0,NaN,12.4,12.4,2,NaN,None,NaN


In [15]:
sf_lines_table[sf_lines_table['RHP_ID__c'] == 'a06VF00000664yNYAQ']

,Id,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastViewedDate,LastReferencedDate,DOS__c,Rev_Code__c,Line_Number__c,Billed_Amount__c,RHP_ID__c,Mod__c,Mod_2nd__c,Units__c,MAR__c,Allowed__c,Client__c,Reconsideration__c,Savings__c,Reductions__c,Bill_Type__c,Exp_Desc__c,HCPCS_CPT_Code__c,Additional_Payment__c,Line_ID__c,Pricing_Method__c,Provider_TIN_Claim__c,Jurisdiction_Claim__c,Pricing_Trigger__c,Exp_Code__c,Exp_Code_2nd__c,MO_QPA_Allowed__c,Exp_Desc_2nd__c,MO_QPA_Reductions__c,MO_Fac_QPA_Reductions__c,MO_Fac_QPA_Allowed__c,MO_Anes_QPA__c,QPA_Allowed__c,MO_Anes_QPA_Reductions__c,QPA_Reductions__c,Unique_Line_Id__c,Code_Edit__c,NDC_Number__c,Target_Amount__c
10090,a0YVF000000ZGzZ2AW,False,L2294116,2024-06-04T17:47:23.000+0000,0058a00000N3AI9AAN,2024-06-04T18:34:18.000+0000,0058a00000N3AI9AAN,2024-06-04T18:34:18.000+0000,None,None,2023-06-02,483,None,12.4,a06VF00000664yNYAQ,SA,25,10.0,13.4,NaN,a048a00000qJMlI,None,100.0,13.4,WORKERS COMP,None,a0b4W00000y1ISnQAM,None,1.0,WORK COMP,a0FVF000000b3oX2AQ,a024W00000HYAjJ,False,None,None,3699.2,None,-3686.8,-11912.9,11925.3,0.0,NaN,12.4,12.4,None,NaN,None,NaN
10091,a0YVF000000ZGza2AG,False,L2294117,2024-06-04T17:47:23.000+0000,0058a00000N3AI9AAN,2024-06-04T17:47:23.000+0000,0058a00000N3AI9AAN,2024-06-04T17:47:23.000+0000,None,None,2023-06-02,483,None,12.4,a06VF00000664yNYAQ,None,None,10.0,13.4,NaN,a048a00000qJMlI,None,100.0,13.4,WORKERS COMP,None,a0b4W00000y1ISnQAM,None,2.0,WORK COMP,a0FVF000000b3oX2AQ,a024W00000HYAjJ,False,None,None,3699.2,None,-3686.8,-11912.9,11925.3,0.0,NaN,12.4,12.4,54179,NaN,None,NaN


In [16]:
sf_lines_table[sf_lines_table['RHP_ID__c'] == 'a06VF0000068hsMYAQ']

,Id,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastViewedDate,LastReferencedDate,DOS__c,Rev_Code__c,Line_Number__c,Billed_Amount__c,RHP_ID__c,Mod__c,Mod_2nd__c,Units__c,MAR__c,Allowed__c,Client__c,Reconsideration__c,Savings__c,Reductions__c,Bill_Type__c,Exp_Desc__c,HCPCS_CPT_Code__c,Additional_Payment__c,Line_ID__c,Pricing_Method__c,Provider_TIN_Claim__c,Jurisdiction_Claim__c,Pricing_Trigger__c,Exp_Code__c,Exp_Code_2nd__c,MO_QPA_Allowed__c,Exp_Desc_2nd__c,MO_QPA_Reductions__c,MO_Fac_QPA_Reductions__c,MO_Fac_QPA_Allowed__c,MO_Anes_QPA__c,QPA_Allowed__c,MO_Anes_QPA_Reductions__c,QPA_Reductions__c,Unique_Line_Id__c,Code_Edit__c,NDC_Number__c,Target_Amount__c
10116,a0YVF000000a8Gn2AI,False,L2294121,2024-06-06T15:20:24.000+0000,0058a00000N3AI9AAN,2024-06-06T15:25:12.000+0000,0058a00000N3AI9AAN,2024-06-06T15:25:12.000+0000,None,None,2023-06-02,483,None,12.4,a06VF0000068hsMYAQ,SA,25,10.0,13.4,NaN,a04Uu0000004LkD,None,100.0,13.4,WORKERS COMP,None,a0b4W00000y1ISnQAM,None,1.0,WORK COMP,a0FVF000000b3oX2AQ,a024W00000HYAjJ,False,None,None,3699.20,None,-3686.80,-11912.90,11925.30,0.0,NaN,12.4,12.4,54178,NaN,None,NaN
10117,a0YVF000000a8Go2AI,False,L2294122,2024-06-06T15:20:24.000+0000,0058a00000N3AI9AAN,2024-06-06T15:25:12.000+0000,0058a00000N3AI9AAN,2024-06-06T15:25:12.000+0000,None,None,2023-06-02,483,None,12.4,a06VF0000068hsMYAQ,None,None,1.0,13.4,NaN,a04Uu0000004LkD,None,100.0,13.4,WORKERS COMP,None,a0b4W00000y1ISnQAM,None,2.0,WORK COMP,a0FVF000000b3oX2AQ,a024W00000HYAjJ,False,None,None,369.92,None,-357.52,-1180.13,1192.53,0.0,NaN,12.4,12.4,54179,NaN,None,NaN


In [ ]:
groupClient_foreign_key = list(table_store['Groups_Clients__c']['Related_Client__c'])


client_table = table_store['Clients__c']
client_table[client_table['Id'].isin(groupClient_foreign_key)]

In [2]:
client_table = table_store['Clients__c']

client_table.columns

Index(['Id', 'OwnerId', 'IsDeleted', 'Name', 'CreatedDate', 'CreatedById',
       'LastModifiedDate', 'LastModifiedById', 'SystemModstamp',
       'LastActivityDate', 'LastViewedDate', 'LastReferencedDate',
       'Prior_Month_Claims__c', 'Prior_Month_Savings__c',
       'Invoice_Frequency__c', 'Appeals_Contact__c', 'Bill_Type__c',
       'Address__c', 'City__c', 'State__c', 'Zip__c', 'Phone__c', 'Email__c',
       'Contact_Name__c', 'Contact_Title__c', 'Logo_URL__c',
       'Appeals_Email__c', 'Prior_Month_Pro_Claims__c',
       'Prior_Month_Inst_Claims__c', 'Prior_Month_Pro_Billed__c',
       'Prior_Month_Pro_Allowed__c', 'Claims_Contact__c',
       'Prior_Month_Allowed__c', 'Prior_Month_Pro_MAR__c',
       'Prior_Month_Pro_Savings__c', 'Prior_Month_Inst_Billed__c',
       'Claims_Email__c', 'Claims_Contact_2__c', 'Claims_Email_2__c',
       'Appeals_Contact_2__c', 'Appeals_Email_2__c', 'Incoming_Claims__c',
       'Outgoing_Claims__c', 'Prior_Month_Inst_MAR__c', 'Invoice_Start__c',


In [40]:
def create_df(source_dict, Source_data, table_store):
    new_columns = [col for mapped_cols in source_dict['MAPPINGS'].values() for col in mapped_cols]
    new_df = pd.DataFrame(columns=new_columns)

    try:
        Table_convert_cols = list(source_dict['TABLE_CONVERT'].keys())
    except:
        Table_convert_cols = []

    print(Table_convert_cols)

    for source_col, target_cols in source_dict['MAPPINGS'].items():
        if source_col in Table_convert_cols:
            map_table = source_dict['TABLE_CONVERT'][source_col]
            print(map_table)
            
            try:
                string_ids = [
                        str(int(float(id))) if pd.notna(id) and id != 'nan' else np.nan
                        for id in Source_data[source_col]                               #### handeling DRG col
                    ]
            except:
                string_ids = [str(id) for id in Source_data[source_col]]
         
            Source_data[source_col] = string_ids #### col update to match same type in foreign table
            foreign_table = list(map_table.keys())[0]
            print(foreign_table)

            merged = Source_data.merge(table_store[foreign_table].loc[:, map_table[foreign_table]], left_on=source_col, right_on=map_table[foreign_table][0], how='left')

            for idx, col in enumerate(map_table[foreign_table], start=0):  # Start from the first column
                if len(map_table[foreign_table]) == 1:
                    # Handle the edge case where only one column exists
                    target_col = target_cols[0]  # Use the first target column
                    print(f"Single column mapping: {col} to {target_col}")
                    new_df[target_col] = merged[col]
                    break  # Exit the loop since there is only one column to map
                elif idx > 0 and idx - 1 < len(target_cols):  # For columns after the first
                    target_col = target_cols[idx - 1]  # Adjust indexing to match target columns
                    print(f"Mapping {col} to {target_col}")
                    new_df[target_col] = merged[col]
                else:
                    print(f"Skipping column {col} or no target column available.")


        if source_col not in Table_convert_cols:
            if source_col in Source_data.columns:
                for target_col in target_cols:
                    new_df[target_col] = Source_data[source_col]
        

    keys = source_dict['KEYS']


    return new_df, keys




def updated_data_pull(object_name, needed_objects):

    object_fields = needed_objects[object_name]
    fields_str = ", ".join(object_fields)
    query = f"SELECT {fields_str} FROM {object_name}"
    results = sf.query_all(query)

    records = results['records']
    df = pd.DataFrame(records).drop(columns='attributes')  # Remove unwanted attributes column
    
    return df

In [ ]:
from datetime import datetime


def format_date(value):
    try:
        # Attempt to parse and format the date
        return datetime.strptime(value, '%m/%d/%Y').strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        # Return the value as-is if it’s not a valid date
        return value



def upsert(new_df, keys):

    object_name = list(keys.keys())[0]
    sf_object = getattr(sf, object_name)
    object_key = list(keys.values())[0]

    # print(object_key)

    for i in range(len(new_df)):
        # print(i)
        record = new_df.iloc[i, :].to_dict()
        # print(record)
        record_upsert = {k: v for k, v in record.items() if k != object_key}
        # Replace NaN with None
        record_upsert = {k: (None if pd.isna(v) else v) for k, v in record_upsert.items()}
        # Format date fields
        record_upsert = {k: format_date(v) if 'Date' in k or 'DOS' in k else v for k, v in record_upsert.items()}

    
        print(record_upsert)
        
        try:
            sf_object.upsert(f"{object_key}/{record[object_key]}", record_upsert)
            # pass
        except Exception as e:
            print(f"Error during upsert: {e}")

        

In [119]:
# map_claims_table.head()

In [120]:
update_claims_object = upsert(map_claims_table, keys)

NameError: name 'format_date' is not defined

In [103]:
Povider_map = DATA_MAPPINGS['Provider_Object_Map']
Account_map = DATA_MAPPINGS["Account_Object_Map"]
Claims_map = DATA_MAPPINGS['Claim_Object_Map']

### First step - populate providers table
map_account_table, _ = create_df(Account_map, Source_data, table_store)
### Second step - populate providers table
map_provider_table, keys = create_df(Povider_map, Source_data, table_store)
### Third step = Upsert data onto SF
update_provider_object = upsert(map_provider_table, keys)
### Fourth step - repull provider table to update claims table
provider_table_repulled = updated_data_pull('Provider_TIN__c', needed_objects)
### Fifth step - Populate Claims table
map_claims_table, keys = create_df(Claims_map, Source_data, table_store)
###










[]
[]
Unique_TIN__c
0
{'Name': '941105628', 'Unique_TIN__c': '941105628', 'Address__c': 'FILE 54602', 'City__c': 'LOS ANGELES', 'State__c': 'CA', 'Zip__c': 900749998, 'Provider_Name__c': 'KAISER FOUNDATION HOSPITALS '}
{'Name': '941105628', 'Address__c': 'FILE 54602', 'City__c': 'LOS ANGELES', 'State__c': 'CA', 'Zip__c': 900749998, 'Provider_Name__c': 'KAISER FOUNDATION HOSPITALS '}
['TIN', 'Patient Group/Policy Number', 'JurisdictionState', 'DRG', 'Billing Provider Taxonomy']
{'Provider_TIN__c': ['Name', 'Id']}
Provider_TIN__c
Skipping column Name or no target column available.
Mapping Id to Provider_TIN__c
{'Groups_Clients__c': ['Group_Number__c', 'Id']}
Groups_Clients__c
Skipping column Group_Number__c or no target column available.
Mapping Id to Group_Client__c
{'Jurisdiction__c': ['Name', 'Id']}
Jurisdiction__c
Skipping column Name or no target column available.
Mapping Id to Jurisdiction__c
{'DRG__c': ['Name', 'Id']}
DRG__c
Skipping column Name or no target column available.
Mapp

In [ ]:
#### verifying upserts

Provider_data_pull = table_store['Provider_TIN__c']
Account_data_pull = table_store['Account']
Claim_data_pull = table_store['Claims__c']
Lines_data_pull = table_store['Line_Items__c']


provider_table_repulled = updated_data_pull('Provider_TIN__c', needed_objects)
claims_table_repulled = updated_data_pull('Claims__c', needed_objects)


print(Claim_data_pull.shape)




##providers pre upsert shape: 10823, post upsert shape: 10853
##claims pre upload shape: 12931, post upsert shape:

(12931, 99)


In [77]:
provider_table_repulled.shape



(12931, 99)

In [ ]:

unique_providers = list(map(str, set(Source_data['TIN'])))

Provider_data_pull[Provider_data_pull['Name'].isin(unique_providers)].shape
len(set(Source_data['TIN']))

59

In [ ]:
##### 29 providers on sandbox, 59 unique providers 

In [56]:
Provider_data_pull.head()

,Id,OwnerId,IsDeleted,Name,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,LastViewedDate,LastReferencedDate,Special_Repricing__c,Contact__c,Phone__c,Fax__c,Email__c,Notes__c,Unique_TIN__c,Address__c,City__c,State__c,Zip__c,Shelby_Co_Contract__c,City_of_Covington_Contract__c,Provider_Name__c,Avg_WC_Savings__c
0,a0F4W00000W1GxyUAF,0054W00000ASrc5QAD,False,000000002,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,2024-12-06T15:27:38.000+0000,2024-12-06T15:27:38.000+0000,False,None,None,None,None,None,000000002,PO Box 362,Royal Oak,MI,480680000,None,None,Provider Miscellaneous,0.0
1,a0F4W00000W1GxzUAF,0054W00000ASrc5QAD,False,000000009,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,000000009,[Replaced Address] 12288,COLUMBUS,GA,319179998,None,None,VISION EMPLOYEE CLAIMS,0.0
2,a0F4W00000W1Gy0UAF,0054W00000ASrc5QAD,False,00000002,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,00000002,None,None,None,None,None,None,MISCELLANEOUS PROVIDER,0.0
3,a0F4W00000W1Gy1UAF,0054W00000ASrc5QAD,False,000000021,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,000000021,[Replaced Address] 12288,COLUMBUS,GA,319179998,None,None,MEDICAL CLAIM,0.0
4,a0F4W00000W1Gy2UAF,0054W00000ASrc5QAD,False,000000022,2020-09-06T02:00:44.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,0054W00000ASrc5QAD,2024-04-13T08:03:39.000+0000,None,None,None,False,None,None,None,None,None,000000022,[Replaced Address] 12288,COLUMBUS,GA,319179998,None,None,DENTAL CLAIMS,0.0


In [72]:
duplicates = Source_data[Source_data.duplicated(subset=['TIN'], keep=False)]



In [62]:
Source_data.head()

,Claim ID,Claim Number,Line ID,TIN,Provider,Address,City,State,Zip,Patient ID,Patient,Date of Birth,Patient Group/Policy Number,DRG,Diag Code 1st,Diag Code 2nd,Diag Code 3rd,Diag Code 4th,Begin DOS,End DOS,Revenue Code,HCPCS/CPT Code,Mod,Mod (2nd),Units,Billed Amount,MAR,IsQPA,Billing Provider Taxonomy,JurisdictionState,4LCodes,NPI,PlaceofService,AnesthesiaUnits,Billtype,BillProvAddress1,BillProvAddress2,BillProvCity,BillProvState,BillProvZip,DRGCode,ClientName
0,A3038B5E4B6C914tksft,4.32E+14,1,941105628,KAISER FOUNDATION HOSPITALS,FILE 54602,LOS ANGELES,CA,900749998,569713835,WATKINS MONIC,8/12/1982,NNMCMC,NaN,M25562,NaN,NaN,NaN,9/26/2024,9/26/2024,320,73564,NaN,NaN,1,987.0,156.72,N,282N00000X,CA,NaN,NaN,13,,13,FILE 54602,,LOS ANGELES,CA,900749998,,NovaNet - MCMC - RHP
1,BE9124A610A9998tksft,CLU20241115325204,1,351166081,JOHNSON MEMORIAL HOSPITAL,1125 W JEFFERSON STREET,FRANKLIN,IN,461312140,QWCR14517,FISHER TERRI,1/11/1974,100600,NaN,M7672,M722,M7732,NaN,10/17/2024,10/17/2024,510,G0463,NaN,NaN,1,65.0,65.00,N,282N00000X,IN,NaN,1.578930e+09,13,,13,1125 W JEFFERSON STREET,,FRANKLIN,IN,461312140,,Lucent - RHP
2,317D671330C4442tksft,U424319281216,1,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,278,C1889,NaN,NaN,1,3.0,3.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
3,317D671330C4442tksft,U424319281216,2,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,300,81025,NaN,NaN,1,54.0,54.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
4,317D671330C4442tksft,U424319281216,3,610461753,DEACONESS HENDERSON HOSPII,PO BOX 638704,CINCINNATI,OH,452638704,FBDS11454,COBB BETHAN,3/12/1984,100660,NaN,Z1211,K635,K648,K2950,10/22/2024,10/22/2024,310,88305,59,NaN,4,1000.0,1000.00,N,282N00000X,OH,NaN,1.295212e+09,13,,13,PO BOX 638704,,CINCINNATI,OH,452638704,,Lucent - RHP
